In [6]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
#pairing data for training sentence transformer
pair_data = pd.read_csv('/content/drive/MyDrive/innovae-revision/data/google-patents/raw/rejected-pairs-grant.csv')

#missing data in priority art filling date and title
pair_data = pair_data.drop_duplicates(subset = ['appl_id','prior_art_no'])\
                     .dropna(subset = ['prior_art_title','prior_art_priority_date'])\
                     .reset_index(drop = True)

In [40]:
#clean claim and title text
pair_data['prior_art_primary_claim'] = pair_data['prior_art_primary_claim'].apply(lambda x: re.sub('^[^a-zA-Z]*', '', x))
pair_data['primary_claim'] = pair_data['primary_claim'].apply(lambda x: re.sub('^[^a-zA-Z]*', '', x))
pair_data['title'] = pair_data['title'].apply(lambda x:x[0]+x[1:].lower())
pair_data['prior_art_title'] = pair_data['prior_art_title'].apply(lambda x:x[0]+x[1:].lower())
pair_data['title'] = pair_data['title'].apply(lambda x: re.sub('^[^a-zA-Z]*', '', x))
pair_data['prior_art_no'] = pair_data['prior_art_no'].astype(str)

#extract filling year
pair_data['prior_art_year'] = pair_data['prior_art_priority_date'].apply(lambda x:str(x)[:4])
pair_data['new_art_year'] = pair_data['filing_date'].apply(lambda x:str(x)[:4])

In [48]:
#training data used in previous language modeing for InnoVAE
model_data = pd.read_csv('/content/drive/MyDrive/innovae-revision/data/google-patents/raw/ai-patents-date-cpc-text.csv')
model_data['patent_id'] = model_data['patent_id'].astype(str)

In [49]:
#matach two dataset on pair_data.prior_art_no = model_data.patent_id
pair_key = pair_data['prior_art_no'].tolist()
model_key = model_data['patent_id'].tolist()
common_key = list(set(pair_key) & set(model_key))

In [66]:
pair_data['set'] = pair_data['prior_art_no'].apply(lambda x: 'val' if x in common_key else 'train')

In [57]:
pair_data = pair_data[pair_data['new_art_year'] > pair_data['prior_art_year']].reset_index(drop = True)

In [60]:
#training text
pair_data['prior_text'] = 'Year: ' + pair_data['prior_art_year'] + '\n\nTitle: ' + pair_data['prior_art_title'] + '\n\nClaim: ' + pair_data['prior_art_primary_claim']
pair_data['new_text'] = 'Year: ' + pair_data['new_art_year'] + '\n\nTitle: ' + pair_data['title'] + '\n\nClaim: ' + pair_data['primary_claim']

In [67]:
pair_data.to_csv('/content/drive/MyDrive/Patent/Data/patent_grant_cleanded_new.csv',index = False)